In [53]:
import pandas as pd
import numpy as np
import missingno as msno
df = pd.read_parquet("./knee-provider.parquet")

#df.info()


In [54]:
## Verwijderen van NA of Missende waarden in componenten van onze Y-waarden in navolging van preprocessing notebook van Dave 
#df = df[df.t1_sucess != 9]
#df = df[df.t1_satisfaction != 9]
#df = df[df != 999]
#df = df.dropna(subset=['oks_t1_score', 'oks_t0_score'])


In [55]:

# Create a boolean mask to filter rows without missing values or specific values
mask_no_missing = df[['t1_sucess', 't1_satisfaction', 'oks_t1_score']].isnull().any(axis=1)
mask_no_values = df[['t1_sucess', 't1_satisfaction']].isin([9]).any(axis=1)
mask_no_missings_or_values = ~(mask_no_missing | mask_no_values)


# Assign the result to a new column 'missing'
df['missing'] = np.where(mask_no_missings_or_values, "No", "Yes")

# Print the DataFrame with the 'missing' column
df.head()

#df.groupby('t1_sucess').size()       #1766 9s
#df.groupby('t1_satisfaction').size() #2037 9s
#df.groupby('oks_t1_score').size() 
#df['oks_t1_score'].isna().sum()       #2579 NaNs
#df['t1_sucess'].isna().sum()          #0 NaNs
#df['t1_satisfaction'].isna().sum()    #0 NaNs

,provider_code,procedure,revision_flag,year,age_band,gender,t0_assisted,t0_assisted_by,t0_symptom_period,t0_previous_surgery,t0_living_arrangements,t0_disability,heart_disease,high_bp,stroke,circulation,lung_disease,diabetes,kidney_disease,nervous_system,liver_disease,cancer,depression,arthritis,t0_mobility,t0_self_care,t0_activity,t0_discomfort,t0_anxiety,t0_eq5d_index_profile,t0_eq5d_index,t1_assisted,t1_assisted_by,t1_living_arrangements,t1_disability,t1_mobility,t1_self_care,t1_activity,t1_discomfort,t1_anxiety,t1_satisfaction,t1_sucess,t1_allergy,t1_bleeding,t1_wound,t1_urine,t1_further_surgery,t1_readmitted,t1_eq5d_index_profile,t1_eq5d_index,oks_eq_5d_index_t1_predicted,t0_eq_vas,t1_eq_vas,oks_eq_vas_t1_predicted,oks_t0_pain,oks_t0_night_pain,oks_t0_washing,oks_t0_transport,oks_t0_walking,oks_t0_standing,oks_t0_limping,oks_t0_kneeling,oks_t0_work,oks_t0_confidence,oks_t0_shopping,oks_t0_stairs,oks_t0_score,oks_t1_pain,oks_t1_night_pain,oks_t1_washing,oks_t1_transport,oks_t1_walking,oks_t1_standing,oks_t1_limping,oks_t1_kneeling,oks_t1_work,oks_t1_confidence,oks_t1_shopping,oks_t1_stairs,oks_t1_score,oks_oks_t1_predicted,missing
0,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,2,1,1,9,9,9,9,9,9,9,9,9,9,1,2,1,2,2,1,21221,0.691,2,9,2,1,9,9,9,9,9,2,1,2,2,2,2,2,2,99999,NaN,NaN,80,999,NaN,0,1,2,2,2,2,0,1,1,2,2,2,17.0,3,3,4,3,3,3,4,2,4,4,4,3,40.0,34.825798,No
1,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,2,2,9,9,9,9,9,9,9,9,9,9,9,1,9,9,9,9,9,99999,NaN,2,9,2,2,1,1,1,1,1,2,1,2,2,2,2,2,2,11111,1.000,NaN,999,95,NaN,1,4,4,4,0,4,1,4,4,3,4,4,37.0,4,4,4,3,4,4,4,3,3,3,4,4,44.0,43.180367,No
2,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,1,2,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,99999,NaN,2,9,1,2,1,1,1,1,1,1,1,9,9,1,9,2,2,11111,1.000,NaN,999,98,NaN,1,4,4,4,2,4,2,3,3,3,3,4,37.0,4,4,4,4,4,4,4,2,4,4,4,4,46.0,42.201862,No
3,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,2,1,9,1,9,9,9,9,9,9,9,9,9,1,9,9,9,9,9,99999,NaN,2,9,2,1,1,1,2,2,1,3,2,2,2,2,2,2,2,11221,0.760,NaN,999,60,NaN,1,2,2,2,2,2,2,0,1,3,3,2,22.0,3,2,4,4,1,4,4,1,3,3,4,3,36.0,36.826900,No
4,ADP02,Knee Replacement,0,2018/19,NaN,NaN,1,0,3,2,2,1,9,1,9,9,9,9,9,9,9,9,9,1,2,1,2,9,1,21291,NaN,2,9,2,1,2,1,2,2,1,2,1,1,2,1,2,2,2,21221,0.691,NaN,75,80,74.157318,0,1,4,2,2,1,0,1,1,0,0,0,12.0,2,2,4,2,2,3,3,2,2,3,2,1,28.0,33.434299,No


In [56]:
#Ik hernoem alle kolommen die we voor de Y gebruiken zodat ik ze niet meefilter als ik alle t1-kolommen drop. 
df = df.rename({'oks_t1_score': 'oks_Y1_score'}, axis=1)
df = df.rename({'t1_sucess': 'Y1_success'}, axis=1)
df = df.rename({'t1_satisfaction': 'Y1_satisfaction'}, axis=1)
#print(dfn0)
df.drop(list(df.filter(regex = 't1')), axis = 1, inplace = True)
df.drop(df['t0_eq5d_index_profile'])

list(df.filter(regex='t1'))
#print(df)

[]

In [57]:
# set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# create the 'missing' column
#df['missing'] = df.isnull().any(axis=1)
#df['missing'] = df.isnull().any(axis=1)

#Tel hoeveel patiënten er in elke groep zitten
df1 = df['missing'].value_counts(dropna=False)
print(df1)
#No     135000
#Yes      4236

# group the data by the 'missing' column and calculate the mean of each column
grouped = df.groupby('missing').mean()

# calculate the absolute difference in means between the two groups
diff = abs(grouped.diff())

# sort the columns based on the difference in means
sorted_columns = diff.loc['Yes'].sort_values(ascending=False).index.tolist()

# reorder the columns in the grouped dataframe
grouped = grouped[sorted_columns]

# print the group means
print(grouped)


#Bij de groep patiënten waar er bij geen van de Y-componenten een missing of 9-waarde wordt gevonden (135000 patiënten),
# is de OKS-score op T1 gemiddeld 36, de satisfaction is er gemiddeld 2.3 en de successcore is er gemiddeld 1.4
#Bij de groep patiënten waar er bij tenminste één van de Y-componenten een missing of 9-waarde wordt gevonden (3866 patiënten), 
# is de OKS-score op T1 gemiddeld 30, de satisfaction 5.9 en de successcore 5. 
# Als tenminste één van de Y-componenten mist, zien we dus een lagere OKS-score op T1, maar een hogere satisfaction en successcore. 
# Wellicht zijn de zeer tevreden patiënten minder geneigd om nogmaals de OKS in te vullen na de operatie? 
# De missings in de missinggroup zijn gebaseerd op: 1766 missings bij Succes T1, 2037 missings bij Satisfaction T1 en 2579 missings bij OKS T1. 


No     135370
Yes      3866
Name: missing, dtype: int64
         t0_eq5d_index_profile   t0_eq_vas  oks_Y1_score  Y1_satisfaction  \
missing                                                                     
No                23182.157324  153.138147     35.942448         2.253838   
Yes               24356.256079  184.647957     30.439783         5.914640   

         Y1_success  oks_t0_score  t0_anxiety  oks_t0_shopping  t0_self_care  \
missing                                                                        
No         1.429371     19.014875    1.678045         1.918313      1.548925   
Yes        4.998707     17.826374    1.907915         1.713658      1.731764   

         circulation  t0_discomfort  t0_living_arrangements  diabetes  \
missing                                                                 
No          8.551806       2.635059                1.347086  8.005215   
Yes         8.383342       2.788670                1.497672  7.870150   

         oks_t0_washi

C:\Users\neelt\AppData\Local\Temp\ipykernel_12568\3230345956.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = df.groupby('missing').mean()


In [9]:
# set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# create the 'missing' column
#df['missing'] = df.isnull().any(axis=1)
#Y1 = ['oks_Y1_score', 'Y1_success', 'Y1_satisfaction']
#df['missing'] = df[Y1].isnull().any(axis=1)

df.head()

,provider_code,procedure,revision_flag,year,age_band,gender,t0_assisted,t0_assisted_by,t0_symptom_period,t0_previous_surgery,t0_living_arrangements,t0_disability,heart_disease,high_bp,stroke,circulation,lung_disease,diabetes,kidney_disease,nervous_system,liver_disease,cancer,depression,arthritis,t0_mobility,t0_self_care,t0_activity,t0_discomfort,t0_anxiety,t0_eq5d_index_profile,t0_eq5d_index,Y1_satisfaction,Y1_success,t0_eq_vas,oks_t0_pain,oks_t0_night_pain,oks_t0_washing,oks_t0_transport,oks_t0_walking,oks_t0_standing,oks_t0_limping,oks_t0_kneeling,oks_t0_work,oks_t0_confidence,oks_t0_shopping,oks_t0_stairs,oks_t0_score,oks_Y1_score,missing
0,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,2,1,1,9,9,9,9,9,9,9,9,9,9,1,2,1,2,2,1,21221,0.691,2,1,80.0,0,1,2,2,2,2,0,1,1,2,2,2,17.0,40.0,False
1,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,2,2,9,9,9,9,9,9,9,9,9,9,9,1,9,9,9,9,9,99999,NaN,2,1,NaN,1,4,4,4,0,4,1,4,4,3,4,4,37.0,44.0,False
2,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,1,2,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,99999,NaN,1,1,NaN,1,4,4,4,2,4,2,3,3,3,3,4,37.0,46.0,False
3,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,2,1,9,1,9,9,9,9,9,9,9,9,9,1,9,9,9,9,9,99999,NaN,3,2,NaN,1,2,2,2,2,2,2,0,1,3,3,2,22.0,36.0,False
4,ADP02,Knee Replacement,0,2018/19,NaN,NaN,1,0,3,2,2,1,9,1,9,9,9,9,9,9,9,9,9,1,2,1,2,9,1,21291,NaN,2,1,75.0,0,1,4,2,2,1,0,1,1,0,0,0,12.0,28.0,False
